# LSTM
- in LSTM info goes through 
    - forget gate (gets prev. hidden state & $x^t1$), 
    - input gate (sigmoid, tanh), 
    - output gate (tanh)
- LSTM can deal with long sequences (robust to vanishing gradients?)
- https://www.youtube.com/watch?v=uSdku8Q3d0A

## LSTM Math
Some symbols look funny...🤔
![lstm_math.png](img/lstm_math.png)
[Source](https://www.coursera.org/learn/sequence-models-in-nlp/supplement/ANbcf/lstm-equations-optional)


# Named Entity Recognition (NER)
- scan text fast for desired info
- find & extract defined entities (e.g. names, offensive vocab)

## NER types
- geographical (Germany)
- organizations (EOS)
- geopolitical (German)
- time (June 2021)
- artifacts (...)
- persons (Angela Merkel)


Labeled Sentence:
*Sebastian (PER) fährt nächste Woche (TIME) nach Erfurt (GEO).*

## Examples Applied NER
- efficient search engines (our confluence needs NER 😁)
- recommendation engines (may not work for all recommendation use cases)
- first level customer communication (call/chat) (e.g. forward customer to the best agent for his request)
- tradobot (1. evaluate scraped content with NER, 2. trade accordingly)